In [ ]:
import os
import re
import pandas as pd
import sys


class Indicator_Handler(object):

    def __init__(self, db_path):
        self._initilize(db_path)

    def _initilize(self, db_path):
        
        self.db_path = db_path
        self._filepaths = self._get_global_filepaths(db_path)
        self._path_n = len(self._filepaths)
        
        coin_names = self._get_coin_names()
        self._name2path = {name: self._filepaths[i]
                           for i, name in enumerate(coin_names)}
        
        print("[Indicator_Handler._initialize] Loading local ohlcv datas...")
        ohlcvs = [self._get_ohlcv(*t) for t in enumerate(coin_names)]
        sys.stdout.write("\r100%% loaded.% 35s"%"\n")
        
        self.name2ohlcv = {name: ohlcvs[i][0]
                           for i, name in enumerate(coin_names)
                           if ohlcvs[i][1]}
        self.coin_names = self.name2ohlcv.keys()
    
    def _get_global_filepaths(self, db_path):
        ls = os.listdir(db_path)
        paths = [os.path.join(db_path, c) for c in ls]
        filepaths = [[path] if os.path.isfile(path)
                     else self._get_global_filepaths(path)
                     for path in paths]
        filepaths = sum(filepaths, [])
        return filepaths
        
    def _get_coin_names(self):

        def get_coin_name(filename):
            text = re.sub(".csv", "", filename)
            text = re.sub("-OHLCV", "", text)
            text = re.sub("-ohlcv", "", text)
            text = re.sub("_SPOT", "", text)
            coin_name = re.sub("_", "-", text)
            # 순서 바꿔줘야함
            return coin_name
        
        filepaths = self._filepaths
        filenames = [os.path.split(path)[-1] for path in filepaths]
        return [get_coin_name(name) for name in filenames]

    def _get_ohlcv(self, i, coin_name):
        
        filepath = self._name2path[coin_name]
        df = pd.read_csv(filepath, index_col=0)
        
        sys.stdout.write("\r% 3d%% loaded... | % -30s"
                         %(int(float(i)/self._path_n*100),
                           coin_name))
        if not len(df):
            return None, False 
        else:
            df.index = df["time"]
            df = df.drop(["time"], axis=1)
            return df, True



DB_PATH = "./data/ohlcv"
ih = Indicator_Handler(DB_PATH)

metadata = {
    "ABANDS": """[Acceleration Bands]
https://www.tradingtechnologies.com/help/x-study/technical-indicator-definitions/acceleration-bands-abands/
""" 
}

func_dic = {
    "ABANDS": None
}